In [4]:
#!/usr/bin/env python
# coding: utf-8
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import keras
print("keras = {}".format(keras.__version__))
import tensorflow as tf
print("tensorflow-gpu = {}".format(tf.__version__))
try:
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
except:
    pass
import random
import shutil
import argparse
import sklearn
from pathlib import Path
from utils import *
from unet3d import *
from config import *

class set_args():
    gpu = 0
    data = None
    apps = 'ncs'
    run = 1
    cv = None
    subsetting = None
    suffix = 'genesis'
    task = 'segmentation'
    
args = set_args()

if args.gpu is not None:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu)
    
from ncs_data import *
conf = ncs_config(args)

conf.display()

keras = 2.2.4
tensorflow-gpu = 1.13.1

Configurations:
arch                           Vnet
batch_size                     16
data                           /mnt/dataset/shared/zongwei/LIDC
exp_name                       Vnet-genesis
input_cols                     64
input_deps                     32
input_rows                     64
logs_path                      models/ncs/run_1/logs
lr                             0.001
max_queue_size                 1
model_path                     models/ncs/run_1
nb_epoch                       10000
optimizer                      adam
patience                       50
verbose                        1
weights                        pretrained_weights/Genesis_Chest_CT.h5
workers                        1




# Train

In [5]:
x_train, y_train = load_image(conf, 'train')
print('x_train: {} | {} ~ {}'.format(x_train.shape, np.min(x_train), np.max(x_train)))
print('y_train: {} | {} ~ {}'.format(y_train.shape, np.min(y_train), np.max(y_train)))

x_valid, y_valid = load_image(conf, 'valid')
print('x_valid: {} | {} ~ {}'.format(x_valid.shape, np.min(x_valid), np.max(x_valid)))
print('y_valid: {} | {} ~ {}'.format(y_valid.shape, np.min(y_valid), np.max(y_valid)))

model = unet_model_3d((1,conf.input_rows,conf.input_cols,conf.input_deps), batch_normalization=True)
if conf.weights is not None:
    print("[INFO] Load pre-trained weights from {}".format(conf.weights))
    model.load_weights(conf.weights)
model, callbacks = model_setup(model, conf, task=args.task)

while conf.batch_size > 1:
    # To find a largest batch size that can be fit into GPU
    try:
        model.fit(x_train, y_train,
                  validation_data=(x_valid, y_valid),
                  batch_size=conf.batch_size,
                  epochs=conf.nb_epoch, 
                  verbose=conf.verbose, 
                  shuffle=True,
                  callbacks=callbacks)
        break
    except tf.errors.ResourceExhaustedError as e:
        conf.batch_size = int(conf.batch_size - 2)
        print("\n> Batch size = {}".format(conf.batch_size))

x_train: (4082, 1, 64, 64, 32) | 0.0 ~ 1.0
y_train: (4082, 1, 64, 64, 32) | 0 ~ 1
x_valid: (3126, 1, 64, 64, 32) | 0.0 ~ 1.0
y_valid: (3126, 1, 64, 64, 32) | 0 ~ 1
[INFO] Load pre-trained weights from pretrained_weights/Genesis_Chest_CT.h5
Train on 4082 samples, validate on 3126 samples
Epoch 1/10000

> Batch size = 14
Train on 4082 samples, validate on 3126 samples
Epoch 1/10000
1036/4082 [======>.......................] - ETA: 7:37 - loss: 0.4391 - mean_iou: 0.6119 - dice_coef: 0.5609
> Batch size = 12
Train on 4082 samples, validate on 3126 samples
Epoch 1/10000
4082/4082 [==============================] - 709s 174ms/step - loss: 0.2932 - mean_iou: 0.7156 - dice_coef: 0.7068 - val_loss: 0.3287 - val_mean_iou: 0.7407 - val_dice_coef: 0.6713

Epoch 00001: val_loss improved from inf to 0.32870, saving model to models/ncs/run_1/Vnet-genesis.h5
Epoch 2/10000
4082/4082 [==============================] - 699s 171ms/step - loss: 0.2394 - mean_iou: 0.7524 - dice_coef: 0.7606 - val_loss: 0.39


Epoch 00025: val_loss did not improve from 0.22408
Epoch 26/10000
4082/4082 [==============================] - 704s 172ms/step - loss: 0.0485 - mean_iou: 0.8667 - dice_coef: 0.9515 - val_loss: 0.2327 - val_mean_iou: 0.8672 - val_dice_coef: 0.7673

Epoch 00026: val_loss did not improve from 0.22408
Epoch 27/10000
4082/4082 [==============================] - 704s 172ms/step - loss: 0.0475 - mean_iou: 0.8678 - dice_coef: 0.9525 - val_loss: 0.2368 - val_mean_iou: 0.8682 - val_dice_coef: 0.7632

Epoch 00027: val_loss did not improve from 0.22408

Epoch 00027: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 28/10000
4082/4082 [==============================] - 704s 172ms/step - loss: 0.0447 - mean_iou: 0.8688 - dice_coef: 0.9553 - val_loss: 0.2344 - val_mean_iou: 0.8693 - val_dice_coef: 0.7656

Epoch 00028: val_loss did not improve from 0.22408
Epoch 29/10000
4082/4082 [==============================] - 704s 172ms/step - loss: 0.0437 - mean_iou: 0.8698 - dice_coef: 

# Test

In [8]:
model = unet_model_3d((1,conf.input_rows,conf.input_cols,conf.input_deps), batch_normalization=True)
print("[INFO] Load trained model from {}".format( os.path.join(conf.model_path, conf.exp_name+".h5") ))
model.load_weights( os.path.join(conf.model_path, conf.exp_name+".h5") )

x_test, y_test = load_image(conf, 'test')
print('x_test: {} | {} ~ {}'.format(x_test.shape, np.min(x_test), np.max(x_test)))
print('y_test: {} | {} ~ {}'.format(y_test.shape, np.min(y_test), np.max(y_test)))

_ = segmentation_model_evaluation(model=model, config=conf, x=x_test, y=y_test, note=conf.exp_name)

[INFO] Load trained model from models/ncs/run_1/Vnet-genesis.h5
x_test: (852, 1, 64, 64, 32) | 0.0 ~ 1.0
y_test: (852, 1, 64, 64, 32) | 0 ~ 1
852/852 [==============================] - 40s 47ms/step
[INFO] Vnet-genesis
x:  (852, 1, 64, 64, 32) | 0.0 ~ 1.0
y:  (852, 1, 64, 64, 32) | 0.0 ~ 1.0
p:  (852, 1, 64, 64, 32) | 0.0 ~ 1.0
[INFO] Dice = 74.39%
[INFO] IoU  = 59.22%
[EVAL] Dice = 75.38%
[EVAL] IoU  = 77.14%
